* 有些起訴書對應的裁判書消失，以 maintext 數量為主
* 把所有非空的起訴書用 Ckiptagger 斷詞

In [11]:
print(len(maintext), len(facts))

214837 214973


In [3]:
# Segmentation

import pickle, json, re, time, logging, os
import tensorflow as tf
from ckiptagger import WS

start_time = time.time()

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) # ignore tensorflow warning
os.environ["CUDA_VISIBLE_DEVICES"] = '0' # to use gpu
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config) # let keras use only required VRAM
ws = WS('./ckip/data')

logging.basicConfig(level=logging.DEBUG,
                    format="%(asctime)s %(message)s",
                    datefmt='%Y-%m-%d %H:%M:%S',
                    handlers=[logging.FileHandler('./log/ckiptagger.log', 'w', encoding='utf-8')])

with open('facts.json', 'r') as f:
    facts = json.load(f)
with open('maintext_parsed.json', 'r') as f:
    maintext = json.load(f)

fact_seg = []
ws = WS('./ckip/data')

for i, key in enumerate(maintext):
    try:
        fact = re.sub('\s', '', facts[key])
        fact_seg.append(' '.join(ws([fact])[0]))
        if i % 2000 == 0:
            logging.debug("已處理 %d " % i)
    except:pass
logging.debug("完成，共處理 %d " % i)

with open('./pkl/fact_seg.pkl', 'wb') as f:
    pickle.dump(fact_seg, f)
with open('./data/fact_seg.txt', 'w', encoding='utf-8') as f:
    for line in fact_seg:
        f.write(line+'\n')
        
logging.debug("--- %s seconds ---" % (time.time() - start_time))
logging.debug('%H:%M:%S', time.gmtime(time.time() - start_time))

* 訓練 gensim word2vec

In [1]:
%%time
import logging
from gensim.models import word2vec

logging.basicConfig(level=logging.DEBUG,
                    format="%(asctime)s %(message)s",
                    datefmt='%Y-%m-%d %H:%M:%S',
                    handlers=[logging.FileHandler('./log/wordvec.log', 'w', encoding='utf-8')])

sentences = word2vec.LineSentence('./data/fact_seg.txt')
model = word2vec.Word2Vec(sentences, size=250, min_count=1, sg=0, iter=100, workers=4, seed=55688)
model.save('./data/fact_w2v_cbow.model')

CPU times: user 3h 35min 17s, sys: 57.5 s, total: 3h 36min 15s
Wall time: 56min 2s
